<a href="https://colab.research.google.com/github/sword-ace/Using-SGNN-for-Depression-Estimate/blob/main/ad_data_preprocess(build_graph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # !pip install transformers
# # !pip install sentencepiece
# !pip install word2vec

In [ ]:
# !pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
# !pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu113.html
# !pip install -q torch-geometric 

In [ ]:
# # torch geometric
# try: 
#     import torch_geometric
# except ModuleNotFoundError:
#     # Installing torch geometric packages with specific CUDA+PyTorch version. 
#     # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details 


In [ ]:
# from google.colab import drive
# # drive.mount('/content/drive')

In [ ]:
# # # # # !rm -rf transd #remove doc
# # # # !rm -rf 
# # # !pip uninstall torch==1.0.1 torchvision==0.2.2
# # # !pip install torchvision == 1.0.61

# !pip uninstall torch-geometricy

In [ ]:
# from transformers import BertModel, BertTokenizer, XLNetModel, XLNetTokenizer

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import PorterStemmer, WordNetLemmatizer
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
import re
import string
import torch
stops = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
#Text cleaning function
def preprocessingText1(text):
  text = text.lower() #text to lowercase
  text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
  text = re.sub(r'<.*?>', '', text) #remove html
  text = re.sub(r'[0-9]+', '', text) #remove number
  text = " ".join([word for word in text.split() if word not in stops]) #remove stopwords
  text = re.sub(r'[^\w\s]', '', text) #remove punctiation
  text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings
  text = re.sub('\[.*?\]', '', text)
  text = re.sub(r'http', '', text)
  text = re.sub(r'www', '', text)
  text = re.sub(r'https', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  for c in ['\r', '\n', '\t'] :
    text = re.sub(c, ' ', text) #replace newline and tab with tabs
  text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text


#Text cleaning function
def preprocessingText(text):
  # text = text.lower() #text to lowercase
  # text = re.sub(r'&lt;', '', text) #remove '&lt;' tag
  # text = re.sub(r'<.*?>', '', text) #remove html
  # text = re.sub(r'[0-9]+', '', text) #remove number
  # text = " ".join([word for word in text.split() if word not in stops]) #remove stopwords
  # text = re.sub(r'[^\w\s]', '', text) #remove punctiation
  # text = re.sub(r'[^\x00-\x7f]', '', text) #remove non ASCII strings
  # text = re.sub('\[.*?\]', '', text)
  # text = re.sub(r'http', '', text)
  # text = re.sub(r'www', '', text)
  # text = re.sub(r'https', '', text)
  # text = re.sub('<.*?>+', '', text)
  # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  # text = re.sub('\n', '', text)
  # text = re.sub('\w*\d\w*', '', text)
  for c in ['\r', '\n', '\t'] :
    text = re.sub(c, ' ', text) #replace newline and tab with tabs
  # text = re.sub('\s+', ' ', text) #replace multiple spaces with one space
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text

In [ ]:
# # for d in data_list:
# n_samples = len(text_train)
# torch.save(preprocessed_text_train,'data'+ f'/catg_{n_samples}_train.pt')
# torch.save(y_train,'data'+ f'/catg_{n_samples}_trlabel.pt')

# # ##load the data
# # torch.load('/content/data/newdep__train.pt')

In [ ]:

# data_test = pd.read_pickle('/content/twit_test.pkl')
# text_test = data_test.clean_T.values

# # text_train = data_train.text.values

# ### setting the one-hot label
# y_train = torch.from_numpy(np.vstack(data_train.Labels.values)).float()
# y_test = torch.from_numpy(np.vstack(data_test.Labels.values)).float()

# print('Train label shape {}'.format(y_train.shape))
# print('Test label shape {}'.format(y_test.shape))

In [ ]:
# # # Get max training sequence length
# # maxlen = max([len(x) for x in train_sequences])


# preprocessed_text_train = [preprocessingText(text) for text in text_train]
# preprocessed_text_test = [preprocessingText(text) for text in text_test]

# print('BEFORE CLEANING: {}'.format(text_train[489]))
# print('AFTER CLEANING: {}'. format(preprocessed_text_train[489]))
# # print('AFTER CLEANING: {}'.format(len(preprocessed_text_train[5]))

In [ ]:
# import torch
import networkx as nx
import torch.nn as nn
from torch import Tensor as Tensor
from torch.nn import Linear as Linear
import torch.nn.init as init
from torch.nn.init import _calculate_correct_fan, calculate_gain
import torch.nn.functional as F
# from torch_geometric.nn import MessagePassing
from torch_geometric import data
import os
import numpy as np

In [ ]:
# import sys, random
#('/content/data/depress_dev.pkl')
data_train = pd.read_pickle('/content/data/depress_train.pkl') #pd.read_pickle('/content/data/depress_dev.pkl') #('/content/data/twit_train.pkl') #('/content/data/posneg_train.pkl' ) #('/content/data/depress_train.pkl') #('/content/data/posneg_train.pkl' ) #pd.read_pickle('/content/data/twit_train.pkl')  #('/content/data/posneg_train.pkl') #('/content/data/twit_train.pkl') #('/content/data/ad_train.pkl')

# text_train = data_train.clean_T.values

text_train = data_train.text

preprocessed_text_train = [preprocessingText(text) for text in text_train]
# print(data_train.Labels.values)

y_train = torch.from_numpy(np.vstack(data_train.Labels.values)).float()

# y_train = np.vstack(data_train.Labels.values.astype(np.float))
print('AFTER CLEANING: {}'. format(preprocessed_text_train[0]))
# print(y_train[500])
# print('AFTER CLEANING: {}'.format(len(preprocessed_text_train[5]))

AFTER CLEANING: i am from here in california oh well that it is big and broad there is a lot to do a lot of job opportunity than other state pretty much that it is big and there is a lot you can do here i do not like traffic maybe the violence rate bad news even though you wan na know what is going on in your environment but you still have to watch it so you can look out to see what is going on on a daily basis sociology i have a two year degree in liberal art but my major wa sociology i am not still working on it i am actually an mta bus operator well since metro is steady growing my dream job is to move up in the company and i am about to actually start some class for supervisory next week so because they look to promote within the company so that is the good thing there is a lot of opportunity at metro where you can take class and they will pay for your tuition and thing of that nature it is not actually hard a right now it is easy if you take advantage of it right now cause they lo

In [ ]:
use_cuda = True
gpu = 0
if use_cuda:
      device = torch.device("cuda:" + str(gpu))
      torch.cuda.set_device(gpu)
      os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
else:
    device = "cpu"
device = device
# args.kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print('Using device:', device)

Using device: cuda:0


In [ ]:
# t  = Tokenizer()
# # Defining 4 document lists
# fit_text = ['Machine Learning Knowledge',
#             'Deep Learning',
#             'Artificial Intelligence']

# t.fit_on_texts(fit_text)
# tt = t.texts_to_sequences(fit_text)
# ttt = torch.from_numpy(pad_sequences(tt, maxlen=5, padding = 'post'))#, truncating='post'))
# print(tt)
# print(ttt)

In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(preprocessed_text_train)

# sequences_text_train = tokenizer.texts_to_sequences(preprocessed_text_train)
# sequences_text_test = tokenizer.texts_to_sequences(preprocessed_text_test)  ##must convert to a list

# len(sequences_text_test[0])
# # sequences_text_test[0]


In [ ]:
# #convert to index
# X_train = torch.from_numpy(pad_sequences(sequences_text_train, maxlen=90, padding = 'post')) #90
# X_test = torch.from_numpy(pad_sequences(sequences_text_test, maxlen=38, padding = 'post' )) #padding = 'post'

# # print(len(X_test ), len(sequences_text_test))

# # X_train = torch.from_numpy(pad_sequences(sequences_text_train))
# # X_test = torch.from_numpy(pad_sequences(sequences_text_test))

# print(len(X_test ), len(sequences_text_test))

# # len(X_test[0])
# print(len(X_test[0]))
# print(len(X_train[0]))

In [ ]:
def content_gen(preprocessed_text_train):
  all_content=[]
  # all_words = []
  for doc in preprocessed_text_train:
    words = doc.split()
    all_content.append(words)
#   print(len(set(words)))
# #   # print(set(words))
#     for word in words:
#         # print(word)
#         all_words.append(word)
  return all_content

In [ ]:
embed_size = 200
 
class GloveTokenizer:
    def __init__(self, filename, unk='<unk>', pad='<pad>'):  #'<pad>'
        self.filename = filename
        self.unk = unk
        self.pad = pad
        self.stoi = dict()
        self.itos = dict()
        self.embedding_matrix = [] #list()
        with open(filename, 'r', encoding='utf8') as f: # Read tokenizer file
            for i, line in enumerate(f):
                values = line.split()
                self.stoi[values[0]] = i
                self.itos[i] = values[0]
                self.embedding_matrix.append([float(v) for v in values[1:]])
        if self.unk is not None: # Add unk token into the tokenizer
            i += 1
            self.stoi[self.unk] = i
            self.itos[i] = self.unk
            self.embedding_matrix.append(np.random.rand(embed_size))
        if self.pad is not None: # Add pad token into the tokenizer
            i += 1
            self.stoi[self.pad] = i
            self.itos[i] = self.pad
            self.embedding_matrix.append(np.zeros(embed_size))

        self.embedding_matrix =np.array(self.embedding_matrix) #.astype(np.float32) # Convert if from double to float for efficiency
       

    def encode(self, sentence):
        if type(sentence) == str:
            sentence = sentence.split(' ')
        elif len(sentence): # Convertible to list
            sentence = list(sentence)
        else:
            raise TypeError('sentence should be either a str or a list of str!')
        encoded_sentence = []
        for word in sentence:
            encoded_sentence.append(self.stoi.get(word, self.stoi[self.unk]))
        return encoded_sentence

    def decode(self, encoded_sentence):
        try:
            encoded_sentence = list(encoded_sentence)
        except Exception as e:
            print(e)
            raise TypeError('encoded_sentence should be either a str or a data type that is convertible to list type!')
        sentence = []
        for encoded_word in encoded_sentence:
            sentence.append(self.itos[encoded_word])
        return sentence

    def embedding(self, encoded_sentence):
        return self.embedding_matrix[np.array(encoded_sentence)]

# array([1.2, "abc"], dtype=float/object) 

# filename = '/content/drive/MyDrive/glove.6B.200d.txt'

# tokenizer = GloveTokenizer(f'/content/drive/MyDrive/glove.6B.{embed_size}d.txt')

# tokenizer = GloveTokenizer(f'/content/drive/MyDrive/data2/glove.6B.{embed_size}d.txt')

In [ ]:
# def word2id(d, word):
    
#     try:
#         result = d[word]
#     except KeyError:
#         result = d['UNK']

#     return result

# def cal_PMI(vocab, content_doc, window_size=20):  #50 20
#     # helper = DataHelper(mode="train")
#     # content, _ = helper.get_content()
    
#     d = dict(zip(vocab, range(len(vocab))))
#     # content = [list(map(lambda x: word2id(vocab,x), doc)) for doc in content_doc]
#     # content = [list(map(lambda word: word2id(d, word),word)) for word in content_doc]
#     # #convert to index
#     # print('content',content)
#     pair_count_matrix = np.zeros((len(vocab), len(vocab)), dtype=int)
#     word_count =np.zeros(len(vocab), dtype=int)
    
#     for i, word in enumerate(content_doc):
            
#             try:
#                 word_count[d[word]] += 1
#             except KeyError:
#                 continue
#             start_index = max(0, i - window_size)
#             end_index = min(len(content_doc), i + window_size)
#             for j in range(start_index, end_index):
#                 if i == j:
#                     continue
#                 else:
#                     target_word = content_doc[j]
#                     try:
#                         pair_count_matrix[d[word], d[target_word]] += 1
#                     except KeyError:
#                         continue
            
#     total_count = np.sum(word_count)
#     word_count = word_count / total_count
#     pair_count_matrix = pair_count_matrix / total_count
#     # print(pair_count_matrix)
#     pmi_matrix = np.zeros((len(vocab), len(vocab)), dtype=float)
#     for i in range(len(vocab)):
#         for j in range(len(vocab)):
#             pmi_matrix[i, j] = np.log(
#                 pair_count_matrix[i, j] / (word_count[i] * word_count[j]) 
#             )
#             if pmi_matrix[i, j] <= 0:
#               continue       
    
#     pmi_matrix = np.nan_to_num(pmi_matrix)
    
#     pmi_matrix = np.maximum(pmi_matrix, 0.0)

#     count = 1
#     edges_mappings = np.zeros((len(vocab), len(vocab)), dtype=int)
#     for i in range(len(vocab)):
#         for j in range(len(vocab)):
#             if pmi_matrix[i, j] != 0:
#                 # edges_weights.append(pmi_matrix[i, j])
#                 edges_mappings[i, j] = count
#                 count += 1
    
#     return edges_mappings

In [ ]:
## vocab is a list containing all unique node id
## content has frequent node ids
### this function is for extracting all unique nodes in each graph rather than using vocab5 dict
# def padding_etract(all_vocab, content_vocab):
    
#     feature_matrix = []
#     pad_vob = []
#     for word in all_vocab:
#         # print("word!!!", content_vocab[word])

#         if word not in content_vocab:
#             pad_vob.append('unk')
#         else:
#             pad_vob.append(word)
#     print("print pad word", pad_vob)

        
      
    # unq = np.unique(np.array(each_vob)).reshape(1,-1).squeeze()
    # print("lllll,unq shape",  unq.shape)

    # return pad_vob


def graphcon(doc_ids, ngram = 4):
    ### could use the vocab5 and consider the get the nodes number from whole training dataset
    # model = word2vec.load('/content/drive/My Drive/data2/glove.6B.200d.txt')
    # feature_matrix = []
    # adj_matrix = []
    
    # node_ids = []
    edges = []
    old_edge_id = []
    # print('doc-ids', doc_ids)
    # local_vocab = np.sort(doc_ids).squeeze()
    local_vocab = np.sort(doc_ids)
    # local_vocab = np.array(doc_ids) #.squeeze()

    # local_vocab =  local_vocab[local_vocab != 0]      
    # # print('local content len', len(local_vocab))
    # if len(local_vocab)==0 :
    #     edges.append([0,0])
    old_to_new = dict(zip(local_vocab, range(len(local_vocab))))
    # print(old_to_new)
    # print(local_vocab)
    
    for index, src_word_old in enumerate(local_vocab):
        src = old_to_new[src_word_old]
        # node_ids.append(local_vocab[index])
        
        for i in range(max(0, index - ngram), min(index + ngram+1, len(local_vocab))):
            dst_word_old = local_vocab[i]
            dst = old_to_new[dst_word_old]

        # for dst_word_old in local_vocab[index:]:
            # dst = old_to_new[dst_word_old]
            edges.append([src, dst])
            # print(src_word_old, dst_word_old )
            # old_edge_id.append(edges_matrix[src_word_old, dst_word_old])

        # self circle
        # edges.append([src, src])
        # old_edge_id.append(edges_matrix[src_word_old, src_word_old])

    edges_n = []
    # edge_id = []
    edges_n.extend(edges)
        
    return edges_n   #, np.array(node_ids), edge_id


In [ ]:
#this is to get total word with freq
def build_vocab(train_dataset, tokenizer):
    stoi = {'<unk>': 0, '<pad>': 1} # Re-index
    itos = {0: '<unk>', 1: '<pad>'} # Re-index
    vocab_count = len(stoi)
    
    # vocab_list = [sentence for sentence in train_dataset.split()]
    vocab_list = [sentence for sentence in train_dataset]
    # print(len(vocab_list))
    
    unique_vocab = []
  
    for vocab in vocab_list:
        unique_vocab.extend(vocab)
    
    unique_vocab = list(set(unique_vocab))
    
    # print(len(unique_vocab))
    # print(unique_vocab)
    for vocab in unique_vocab:
        if vocab in tokenizer.stoi.keys():
            stoi[vocab] = vocab_count
            itos[vocab_count] = vocab
            vocab_count += 1
    #### ---------get the vocab glove embedding ------#####
    # embedding_matrix  = None
    embedding_matrix = tokenizer.embedding(tokenizer.encode(list(stoi.keys())))
    return vocab_count, stoi, embedding_matrix , itos

In [ ]:

# content_all = content_gen(preprocessed_text_train)
# vocab_count,stoic, embedding_matrix  =  build_vocab(content_all, tokenizer)

# print(vocab_count, len(node_sets),embedding_matrix.shape )

In [ ]:
# list(stoic.values())

In [ ]:
# stoic.get('enjoy',0)
# len(stoic)
# stoic.get('xbox',0)
# # vocab_count,stoic, embedding_matrix  =  build_vocab(preprocessed_text_train[0], tokenizer)
# # node_sets = [[stoic.get(vocab, 0) for vocab in sentence] for sentence in content_all] 
# # print(vocab_count, len(node_sets),embedding_matrix.shape )

In [ ]:
rand_embedding_matrix = np.random.uniform(-1,1, (20000,128))

def embed_etract(vocab):
    feature_matrix = []
    
    for word_id in vocab:
        # feature_matrix.append(embedding_matrix[word_id])
        feature_matrix.append(rand_embedding_matrix[word_id])      
    return np.array(feature_matrix)

In [ ]:
def create_neighbor_set(node_set, p=4):
    # if type(node_set[0]) != int:
    #     raise ValueError('node_set should be a 1D list!')
    # if p < 0:
    #     raise ValueError('p should be an integer >= 0!')
        
    sequence_length = len(node_set)
    neighbor_set = []

    for i in range(sequence_length):
        neighbor = []
        for j in range(-p, p+1):
            if 0 <= i + j < sequence_length:
                neighbor.append(node_set[i+j])
        neighbor_set.append(neighbor)
    return neighbor_set

def build_nodes(train_doc, stoic): # will also build self.edge_stat and self.public_edge_mask
    # preparing self.train_dataset
    nn =[sentence for sentence in train_doc]
    node_sets= []
    node_vob = []
    for vocab in nn:
        # print(vocab)
        node_vob.append(vocab)
        node_sets.append(stoic.get(vocab, 0)) #0 represents the position of the value of dict

    # node_sets = [[stoic.get(vocab, 0) for vocab in sentence] for sentence in train_doc]
    print("now the train set contains node sets:", node_sets, len(node_sets))
    # neighbor_sets = [create_neighbor_set(node_sets, p=2)]
    # neighbor_sets = [create_neighbor_set(node_set, p=2) for node_set in node_sets]
    return node_sets, node_vob  #,neighbor_sets

def build_bags(doc, stoic): # will also build self.edge_stat and self.public_edge_mask
    # preparing self.train_dataset
    
    node_sets= []
    
    for vocab in doc:
        # print(vocab)
        # node_vob.append(vocab)
        node_sets.append(stoic.get(vocab, 0)) #0 represents the position of the value of dict

    # node_sets = [[stoic.get(vocab, 0) for vocab in sentence] for sentence in train_doc]
    # print("now the train set contains node sets:", node_sets, len(node_sets))
    # neighbor_sets = [create_neighbor_set(node_sets, p=2)]
    # neighbor_sets = [create_neighbor_set(node_set, p=2) for node_set in node_sets]
    return node_sets  #,neighbor_sets

def build_contents(train_doc, stoic, itos): # will also build self.edge_stat and self.public_edge_mask
    # preparing self.train_dataset
    
    content_sets = []
    node_vob =[]
    nn =[sentence for sentence in train_doc]
    for vocab in nn:
        # print(vocab)
        node_vob.append(itos.get(vocab))
        content_sets.append(stoic.get(vocab, 0)) #0 represents the position of the value of dict

    # node_sets = [[stoic.get(vocab, 0) for vocab in sentence] for sentence in train_doc]
    # print("now the train set contains node sets:", content_sets , len(content_sets ), len(nn))
    return content_sets , node_vob

In [ ]:
# def build_public_edge_mask(node_sets, neighbor_sets, min_freq=2):
#     vocab_count =16714
#     edge_stat = torch.zeros(vocab_count, vocab_count)
#     for node_set, neighbor_set in zip(node_sets, neighbor_sets):
#         for neighbor in neighbor_set:
#             for to_node in neighbor:
#                 edge_stat[node_set, to_node] += 1
#     public_edge_mask = edge_stat < min_freq # mark True at uncommon edges
#     return edge_stat, public_edge_mask

In [ ]:
# nd  = build_nodes(content_all[3], stoic)
# # eds, peds = build_public_edge_mask(nd, ed, min_freq=2)
# # eds[0][:400]


In [ ]:
# def graph_data(all_content, stoic,itos, x_train, y_label):
    
#     # pmi, edges_matrix, edg_nums = cal_PMI(window_size=15)
#     # index = 0
#     data_list = []
#     text_list = []
#     full_text = []
#     # ########--------this part keep the node sets are not updated by each content-------_###########
#     # self_vocab =[]
#     # with open(os.path.join('data', 'vocab_5.txt')) as f:
#     #     vocab = f.read()
#     #     self_vocab= vocab.split('\n')
#     # # print(len(self_vocab))
#     # node_sets = build_bags(self_vocab, stoic)

#     for doc_id in range(len(all_content)):
      
#         vocab_get= all_content[doc_id]
        
#         # # # ##filter out by word account
#         # if len(vocab_get) < 2:
#         #     continue
      
#         node_sets, node_vobs = build_nodes(vocab_get, stoic)
#         unq_vob = set(node_sets)
        

#         content_sets, nodes = build_contents(unq_vob, stoic, itos)
#         print(len(nodes), len(unq_vob))

#         text_list.append(nodes)
#         full_text.append(vocab_get)

#         # unq_content = set(vocab_get)
#         # unq_content = set(content)
#         # print('raw transcript lenth', len(content), len(unq_content))
#         # print("content lenth in vocab5", len(vocab_c))
#         print("content vocab in each trans", vocab_get)
#         print("uniq content lenth in each trans", len(set(node_sets)), len(node_sets))
#         # print(unq_content)

#         # content_all = content_gen(preprocessed_text_test)    

#         # ###the len is restricted to 128
#         # unq_content = set(all_content[doc_id][:128])
#         # f = embed_etract(node_sets)
#         f = embed_etract(unq_vob)

#         # f = feature_etract(node_vobs)
#         #  # edges_matrix = cal_PMI(unq_content, content_all[doc_id], window_size=15)
#         # new_pads = np.zeros((128,200))
#         # for ind, value in enumerate(f):
#         #      new_pads[ind] = value
#         # print("the padded vocab size", new_pads.shape)
#         ###----------this is for the original operating mode  --------#########
#         # label =  torch.tensor(y_label[doc_id])

#         ##### --------- ---------------  #######
#         print('file no', doc_id)
#         # index += 1
#        ##--------------------------------------------------##

#         # e  = graphcon(x_train[doc_id])
#         # e  = graphcon(content_sets[:89])
#         e  = graphcon (content_sets, ngram = 8) #8
#         # print("now the train set contains:", content_sets ,len(content_sets))
#         # print("now the train set contains:", x_train[doc_id])
#        ##--------------------------------------------------##
#         edges1 = [np.array([edge[0], edge[1]]) for edge in e]
#         edge_index = torch.tensor(np.array(edges1).T, dtype=torch.long) #.cuda()
#         # edge_attr = torch.tensor(seq_edge_w[edg_ar], dtype = torch.float)   #.cuda()
      
#         # print("edge atr", edge_attr.size())
#         # edge_index, _ = add_remaining_self_loops (edge_index, edge_attr)
#         print("edge index size", edge_index[0].size())
#         print("edge index size2", edge_index[1].size())
#         ####-------------------------------###
#         # ft = torch.tensor(f)
#         ft = torch.tensor(f, dtype=torch.float32) #.cuda()
#         y  = torch.tensor(y_label[doc_id], dtype= torch.float)
#         print("the label!!", y)
#         data_list.append(data.Data(x=ft, edge_index= edge_index, y=y))
    
#     return  data_list, text_list, full_text

In [ ]:
def graph_data(all_content, stoic,itos, x_train, y_label):
    
    # pmi, edges_matrix, edg_nums = cal_PMI(window_size=15)
    # index = 0
    data_list = []
    text_list = []
    full_text = []
    # ########--------this part keep the node sets are not updated by each content-------_###########
    # self_vocab =[]
    # with open(os.path.join('data', 'vocab_5.txt')) as f:
    #     vocab = f.read()
    #     self_vocab= vocab.split('\n')
    # # print(len(self_vocab))
    # node_sets = build_bags(self_vocab, stoic)

    for doc_id in range(len(all_content)):
      
        vocab_get= all_content[doc_id]
        
        # # ##filter out by word account
        if len(vocab_get) < 2:
            continue
      
        node_sets, node_vobs = build_nodes(vocab_get, stoic)
        unq_vob = set(node_sets)
        

        content_sets, nodes = build_contents(unq_vob, stoic, itos)
        print(len(nodes), len(unq_vob))

        text_list.append(nodes)
        full_text.append(vocab_get)

        # unq_content = set(vocab_get)
        # unq_content = set(content)
        # print('raw transcript lenth', len(content), len(unq_content))
        # print("content lenth in vocab5", len(vocab_c))
        print("content vocab in each trans", vocab_get)
        print("uniq content lenth in each trans", len(set(node_sets)), len(node_sets))
        # print(unq_content)

        # content_all = content_gen(preprocessed_text_test)    

        # ###the len is restricted to 128
        # unq_content = set(all_content[doc_id][:128])
        # f = embed_etract(node_sets)
        f = embed_etract(unq_vob)

        # f = feature_etract(node_vobs)
        #  # edges_matrix = cal_PMI(unq_content, content_all[doc_id], window_size=15)
        # new_pads = np.zeros((128,200))
        # for ind, value in enumerate(f):
        #      new_pads[ind] = value
        # print("the padded vocab size", new_pads.shape)
        ###----------this is for the original operating mode  --------#########
        # label =  torch.tensor(y_label[doc_id])

        ##### --------- ---------------  #######
        print('file no', doc_id)
        # index += 1
       ##--------------------------------------------------##

        # e  = graphcon(x_train[doc_id])
        # e  = graphcon(content_sets[:89])
        e  = graphcon (content_sets, ngram = 8) #8
        # print("now the train set contains:", content_sets ,len(content_sets))
        # print("now the train set contains:", x_train[doc_id])
       ##--------------------------------------------------##
        edges1 = [np.array([edge[0], edge[1]]) for edge in e]
        edge_index = torch.tensor(np.array(edges1).T, dtype=torch.long) #.cuda()
        # edge_attr = torch.tensor(seq_edge_w[edg_ar], dtype = torch.float)   #.cuda()
      
        # print("edge atr", edge_attr.size())
        # edge_index, _ = add_remaining_self_loops (edge_index, edge_attr)
        print("edge index size", edge_index[0].size())
        print("edge index size2", edge_index[1].size())
        ####-------------------------------###
        # ft = torch.tensor(f)
        ft = torch.tensor(f, dtype=torch.float32) #.cuda()
        y  = torch.tensor(y_label[doc_id], dtype= torch.float)
        print("the label!!", y)
        data_list.append(data.Data(x=ft, edge_index= edge_index, y=y))
    
    return  data_list, text_list, full_text

In [ ]:
# def graph_data(all_content, stoic, y_label):
    
#     # pmi, edges_matrix, edg_nums = cal_PMI(window_size=15)
#     # index = 0
#     data_list = []
    
#     # seq_edge_w = torch.zeros((edg_nums,1))
#     self_vocab =[]
    
#     # with open(os.path.join('data', 'vocab_5.txt')) as f:
#     #     vocab = f.read()
#     #     self_vocab= vocab.split('\n')
#     # print(len(self_vocab))


#     for doc_id in range(len(all_content)):
      
#         vocab_get= all_content[doc_id]
#         if len(vocab_get) == 0:
#             continue
#         node_sets, node_vobs = build_nodes(vocab_get, stoic)
      
#         # unq_content = set(vocab_get)
#         # unq_content = set(content)
#         # print('raw transcript lenth', len(content), len(unq_content))
#         # print("content lenth in vocab5", len(vocab_c))
#         print("content vocab in each trans", len(node_sets))
#         # print("uniq content lenth in each trans", len(set(node_sets)), node_sets)
#         # print(unq_content)

#         # content_all = content_gen(preprocessed_text_test)    

#         # ###the len is restricted to 128
#         # unq_content = set(all_content[doc_id][:128])
#         f = embed_etract(node_sets)
#         # f = feature_etract(node_vobs)
#         #  # edges_matrix = cal_PMI(unq_content, content_all[doc_id], window_size=15)
#         # new_pads = np.zeros((128,200))
#         # for ind, value in enumerate(f):
#         #      new_pads[ind] = value
#         # print("the padded vocab size", new_pads.shape)
#         ###----------this is for the original operating mode  --------#########
#         # label =  torch.tensor(y_label[doc_id])

#         ##### --------- ---------------  #######
#         print('file no', doc_id)
#         # index += 1

#         e  = graphcon(node_sets)
        
#        ##--------------------------------------------------##
#         edges1 = [np.array([edge[0], edge[1]]) for edge in e]
#         edge_index = torch.tensor(np.array(edges1).T, dtype=torch.long) #.cuda()
#         # edge_attr = torch.tensor(seq_edge_w[edg_ar], dtype = torch.float)   #.cuda()
      
#         # print("edge atr", edge_attr.size())
#         # edge_index, _ = add_remaining_self_loops (edge_index, edge_attr)
#         print("edge index size", edge_index[0].size())
#         print("edge index size2", edge_index[1].size())
#         ####-------------------------------###
#         # ft = torch.tensor(f)
#         ft = torch.tensor(f, dtype=torch.float32) #.cuda()
#         y  = torch.tensor(y_label[doc_id], dtype= torch.float)
#         data_list.append(data.Data(x=ft, edge_index= edge_index, y=y))
    
#     return  data_list

In [ ]:
# #####-----------this is for setting all voca based on train set--------#########
# content_all = content_gen(preprocessed_text_train)
# vocab_count,stoic, embedding_matrix  =  build_vocab(content_all, tokenizer)
# # stoic.get('pad',0)

In [ ]:

# content_all = content_gen(preprocessed_text_train)
# vocab_count,stoic, embedding_matrix  =  build_vocab(content_all, tokenizer)
# print(vocab_count, len(node_sets),embedding_matrix.shape )

# preprocessed_text_train=[]

# with open(os.path.join('data', 'train-stemmed.txt')) as f:
#         vocab = f.read()
#         preprocessed_text_train= vocab.split('\n')

# print("the leng of all doc", len(preprocessed_text_train))



content_train = content_gen(preprocessed_text_train)
vocab_count,stoic, embedding_matrix, itos  =  build_vocab(content_train, tokenizer)
# data_list = graph_data(content_train, stoic, X_train=None, y_train)

data_list, text_list, full_text = graph_data(content_train, stoic,itos, None,y_train)

# content_test = content_gen(preprocessed_text_test)
# data_list = graph_data(content_test, stoic, X_test, y_test)

now the train set contains node sets: [3320, 4775, 2729, 1471, 2299, 656, 4092, 1404, 2416, 133, 606, 1295, 2224, 1309, 1281, 606, 2889, 699, 1364, 2628, 2889, 699, 2606, 3721, 2453, 987, 1049, 660, 2833, 2943, 2416, 133, 606, 1295, 2224, 1281, 606, 2889, 699, 1767, 5111, 2628, 1471, 3320, 2628, 2917, 1641, 146, 2787, 4240, 1965, 2826, 603, 609, 1831, 4051, 1767, 561, 3797, 3319, 1527, 606, 4148, 1185, 2299, 1493, 1393, 241, 1767, 1371, 3705, 1364, 934, 133, 510, 1767, 5111, 4938, 4422, 1364, 4497, 1527, 606, 4148, 1185, 1185, 2889, 2014, 1480, 366, 3320, 3705, 2889, 1729, 2339, 4134, 2299, 3752, 1666, 241, 2185, 3939, 3471, 366, 3320, 4775, 2917, 1371, 4030, 1185, 133, 3320, 4775, 63, 1339, 2730, 681, 1891, 1404, 2592, 4109, 606, 3831, 3118, 2185, 556, 3721, 606, 1364, 367, 3595, 2299, 4240, 212, 2224, 3320, 4775, 289, 1364, 63, 1994, 2215, 4727, 3707, 4830, 2015, 2171, 510, 5064, 4704, 4938, 1364, 2723, 641, 4240, 212, 510, 2416, 606, 4240, 4325, 1208, 1281, 606, 2889, 699, 2606, 245

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


edge index size torch.Size([5572])
edge index size2 torch.Size([5572])
the label!! tensor([2.])
now the train set contains node sets: [3320, 367, 1364, 1113, 3398, 2339, 1169, 3707, 2055, 229, 3320, 2628, 86, 3320, 4775, 2299, 2136, 3721, 3320, 852, 3305, 1364, 403, 1442, 2971, 2889, 2339, 3320, 1641, 4240, 1290, 4240, 3545, 3050, 2162, 128, 3320, 2628, 2917, 1641, 4240, 2215, 2606, 4240, 3315, 2215, 2606, 4240, 4712, 4240, 3315, 4726, 4933, 478, 4712, 4704, 4726, 2215, 2606, 2676, 4726, 2917, 4933, 2212, 5118, 1404, 1564, 133, 3471, 4933, 1105, 1364, 4382, 4975, 1364, 3870, 2299, 1113, 3320, 3705, 2889, 2889, 4960, 4134, 2224, 2889, 1666, 4134, 5071, 2185, 556, 3721, 606, 4030, 3707, 4240, 2410, 4661, 2198, 510, 3320, 625, 5112, 1525, 133, 3320, 4775, 4933, 4520, 1364, 2185, 4286, 3320, 2628, 2917, 2521, 4450, 1111, 5064, 3320, 1641, 3315, 2224, 3320, 1641, 2477, 2075, 3315, 3320, 1641, 3556, 2461, 3680, 1454, 3320, 4775, 2849, 2089, 3315, 3598, 4726, 2917, 2875, 289, 3585, 1552, 133,

In [ ]:

# # print(vocab_count, len(node_sets),embedding_matrix.shape )
# content_train = content_gen(preprocessed_text_train)
# data_list = graph_data(content_train, stoic, y_train)

In [ ]:
data_list[:-20]

[Data(x=[428, 128], edge_index=[2, 7204], y=[1]),
 Data(x=[332, 128], edge_index=[2, 5572], y=[1]),
 Data(x=[579, 128], edge_index=[2, 9771], y=[1]),
 Data(x=[326, 128], edge_index=[2, 5470], y=[1]),
 Data(x=[330, 128], edge_index=[2, 5538], y=[1]),
 Data(x=[274, 128], edge_index=[2, 4586], y=[1]),
 Data(x=[332, 128], edge_index=[2, 5572], y=[1]),
 Data(x=[251, 128], edge_index=[2, 4195], y=[1]),
 Data(x=[251, 128], edge_index=[2, 4195], y=[1]),
 Data(x=[281, 128], edge_index=[2, 4705], y=[1]),
 Data(x=[248, 128], edge_index=[2, 4144], y=[1]),
 Data(x=[247, 128], edge_index=[2, 4127], y=[1]),
 Data(x=[324, 128], edge_index=[2, 5436], y=[1]),
 Data(x=[382, 128], edge_index=[2, 6422], y=[1]),
 Data(x=[264, 128], edge_index=[2, 4416], y=[1]),
 Data(x=[487, 128], edge_index=[2, 8207], y=[1]),
 Data(x=[235, 128], edge_index=[2, 3923], y=[1]),
 Data(x=[305, 128], edge_index=[2, 5113], y=[1]),
 Data(x=[487, 128], edge_index=[2, 8207], y=[1]),
 Data(x=[229, 128], edge_index=[2, 3821], y=[1]),


In [ ]:
text_list[:10]

[['keeping',
  'metro',
  'hopefully',
  'around',
  'degree',
  'ahead',
  'at',
  'think',
  'went',
  'taught',
  'plus',
  'anybody',
  'going',
  'argued',
  'temper',
  'somewhere',
  'actually',
  'happened',
  'trained',
  'needed',
  'thought',
  'idea',
  'kid',
  'support',
  'tendency',
  'own',
  'week',
  'hm',
  'sometimes',
  'it',
  'leave',
  'my',
  'moral',
  'expensive',
  'son',
  'the',
  'traffic',
  'court',
  'moment',
  'interested',
  'learning',
  'mistake',
  'setting',
  'some',
  'bath',
  'and',
  'calm',
  'grandkids',
  'family',
  'sane',
  'depends',
  'situation',
  'company',
  'be',
  'couple',
  'good',
  'now',
  'whole',
  'open',
  'but',
  'in',
  'daughter',
  'quite',
  'meditate',
  'through',
  'already',
  'same',
  'individual',
  'get',
  'hearing',
  'about',
  'year',
  'did',
  'seven',
  'type',
  'had',
  'doing',
  'everybody',
  'out',
  'remember',
  'happen',
  'myself',
  'kept',
  'shoe',
  'sociology',
  'move',
  'that',


In [ ]:
# for d in data_list:
n_samples = len(data_list)
torch.save(data_list,'data'+ f'/renew_{n_samples}_train.pt')

In [ ]:
# for d in data_list:
t_samples = len(text_list)
torch.save(text_list,'data'+ f'/renew_{t_samples}_train_text.pt')

In [ ]:
# for d in data_list:
f_samples = len(full_text)
torch.save(full_text,'data'+ f'/renew_{f_samples}_train_full_text.pt')

In [ ]:
full_text[0]

['i',
 'am',
 'from',
 'here',
 'in',
 'california',
 'oh',
 'well',
 'that',
 'it',
 'is',
 'big',
 'and',
 'broad',
 'there',
 'is',
 'a',
 'lot',
 'to',
 'do',
 'a',
 'lot',
 'of',
 'job',
 'opportunity',
 'than',
 'other',
 'state',
 'pretty',
 'much',
 'that',
 'it',
 'is',
 'big',
 'and',
 'there',
 'is',
 'a',
 'lot',
 'you',
 'can',
 'do',
 'here',
 'i',
 'do',
 'not',
 'like',
 'traffic',
 'maybe',
 'the',
 'violence',
 'rate',
 'bad',
 'news',
 'even',
 'though',
 'you',
 'wan',
 'na',
 'know',
 'what',
 'is',
 'going',
 'on',
 'in',
 'your',
 'environment',
 'but',
 'you',
 'still',
 'have',
 'to',
 'watch',
 'it',
 'so',
 'you',
 'can',
 'look',
 'out',
 'to',
 'see',
 'what',
 'is',
 'going',
 'on',
 'on',
 'a',
 'daily',
 'basis',
 'sociology',
 'i',
 'have',
 'a',
 'two',
 'year',
 'degree',
 'in',
 'liberal',
 'art',
 'but',
 'my',
 'major',
 'wa',
 'sociology',
 'i',
 'am',
 'not',
 'still',
 'working',
 'on',
 'it',
 'i',
 'am',
 'actually',
 'an',
 'mta',
 'bus',
 'o

In [ ]:
a= ?

SyntaxError: ignored

In [ ]:
# for ind in range(len(content_all[:20])):
#   nd, ed = prepare_dataset(content_all[ind], stoic)
#   print(ed)

In [ ]:
# def graph_data(all_content, x_train, y_label):
    
#     # pmi, edges_matrix, edg_nums = cal_PMI(window_size=15)
#     # index = 0
#     data_list = []
    
#     # seq_edge_w = torch.zeros((edg_nums,1))
#     self_vocab =[]
    
#     with open(os.path.join('data', 'vocab_5.txt')) as f:
#         vocab = f.read()
#         self_vocab= vocab.split('\n')
#     print(len(self_vocab))


#     for doc_id in range(len(all_content)):
      
#         vocab_get= all_content[doc_id]
#         content = x_train[doc_id]

#         ###the len is restricted to 128
#         # unq_content = content
#         unq_content = self_vocab    #set(all_content[doc_id][:128]) #[:128]
#         # unq_content = set(vocab_get)
#         # unq_content = set(content)
#         print('raw transcript lenth', len(content), len(unq_content))
#         # print("content lenth in vocab5", len(vocab_c))
#         print("content vocab in each trans", len(vocab_get))
#         print("uniq content lenth in each trans", len(unq_content))
#         # print(unq_content)

#         # content_all = content_gen(preprocessed_text_test)    

#         # ###the len is restricted to 128
#         # unq_content = set(all_content[doc_id][:128])
        
#         f = feature_etract(unq_content)
#         #  # edges_matrix = cal_PMI(unq_content, content_all[doc_id], window_size=15)
#         # new_pads = np.zeros((128,200))
#         # for ind, value in enumerate(f):
#         #      new_pads[ind] = value
#         # print("the padded vocab size", new_pads.shape)
#         ###----------this is for the original operating mode  --------#########
#         # label =  torch.tensor(y_label[doc_id])

#         ##### --------- ---------------  #######
#         print('file no', doc_id)
#         # index += 1

#         e,n, edg_ar = graphcon(content)
        
#        ##--------------------------------------------------##
#         edges1 = [np.array([edge[0], edge[1]]) for edge in e]
#         edge_index = torch.tensor(np.array(edges1).T, dtype=torch.long) #.cuda()
#         # edge_attr = torch.tensor(seq_edge_w[edg_ar], dtype = torch.float)   #.cuda()
      
#         # print("edge atr", edge_attr.size())
#         # edge_index, _ = add_remaining_self_loops (edge_index, edge_attr)
#         print("edge index size", edge_index[0].size())
#         print("edge index size2", edge_index[1].size())
#         ####-------------------------------###

#         ft = torch.tensor(f, dtype=torch.float) #.cuda()
#         y  = torch.tensor(y_label[doc_id], dtype= torch.float)
#         data_list.append(data.Data(x=ft, edge_index= edge_index, y=y))
    
#     return  data_list

In [ ]:
# def build_vocabs(content, min_count=500):   ##250 #300 #350
#         vocab = []

#         for c in content:
#             words = c.split()
#             for word in words:
            
#                 if word not in vocab:
#                     vocab.append(word)

#         freq = dict(zip(vocab, [0 for i in range(len(vocab))]))

#         for c in content:
#             words = c.split()
#             for word in words:
#                 if word not in '':
#                     freq[word] += 1

#         results = []
#         for word in freq.keys():
#             if freq[word] < min_count:
#                 continue
#             else:
#                 results.append(word)

#         results.insert(0, 'unk')
#         with open(os.path.join('data', 'vocab_5.txt'), 'w') as f:
#             f.write('\n'.join(results))
#         # print("the orginal vocab_5 len:{:2d}the current len is:{:2d}".format(len(results),len(self.vocab)))

In [ ]:
# build_vocabs(preprocessed_text_train)

In [ ]:
# preprocessed_text_train preprocessed_text_test
# X_train X_test
# y_train y_test

# content_all = content_gen(preprocessed_text_test)
# data_list = graph_data(content_all, X_test, y_test)


content_all = content_gen(preprocessed_text_train[:1000])
data_list = graph_data(content_all, X_train, y_train)

In [ ]:
# content_all = content_gen(preprocessed_text_train)
# data_list = graph_data(content_all[:20], X_train, y_train)
print(len(data_list))
data_list 

In [ ]:
# for d in data_list:
n_samples = len(data_list)
torch.save(data_list,'data'+ f'/catg_{n_samples}_train.pt')

# ##load the data
# torch.load('/content/data/newdep__train.pt')